# Tartu Ülikooli õppeainete tagasiside meelsusanalüüs
## Autor: Regita Luukas

See kood on valminud 2023. aastal tehtud Tartu Ülikooli Informaatika õppekava bakalaureuse töö raames teemal "Tartu Ülikooli õppeainete tagasiside meelsusanalüüs". 


### 1. Andmete ettevalmistus

In [ ]:
# Ühendamine Google Colabi jaoks
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/My Drive/Colab Notebooks/")

In [ ]:
# Teekide import
!pip install estnltk==1.7.1
import pandas as pd
import numpy as np
import estnltk
from estnltk import Text
from estnltk.taggers import ConllMorphTagger
from estnltk.taggers import MaltParserTagger

In [ ]:
#tagasiside faili import, kommentaari sisaldava lahtri pealkiri peab algoritmi jaoks olema 'tagasiside'
df = pd.read_excel("tagasiside_fail.xlsx")

In [4]:
#valentsileksikoni import
df2 = pd.read_excel("valentsileksikon.xlsx", header=None)
leksikon = df2.rename({0: 'valjend', 1: 'hinnang'}, axis=1)

### 2. Soovituste ja kirjelduste eraldus

In [9]:
tulemus = np.empty(len(df), dtype=object) #Fikseeritud suurusega (korpuse pikkusega) järjend tulemuste hoidmiseks

for i in range(len(df)):
#vaadeldakse ükshaaval läbi kõik andmeread
  kommentaar = df["tagasiside"][i]
  tekst = Text(kommentaar).tag_layer(['words', 'morph_analysis_est']) #morfoloogiline analüüs
  if kommentaar.lower() == "edu!" or kommentaar.lower() == "edu" or kommentaar.lower() == "edu.":
  #kui kommentaar = "edu", siis loetakse soovituseks
    tulemus[i] = ("soovitus")
    continue
  for lemmad in tekst.lemma:
  #kui sisaldab sõna "soovitama" või "soovima", siis loetakse soovituseks
    if "soovitama" in lemmad or "soovima" in lemmad:
      tulemus[i] = ("soovitus")
      break
  if tulemus[i] == ("soovitus"):
    continue
  vormid = list(tekst.morph_analysis_est.vormi_nimetus) #kõigi sõnade sõnavormide list
  for j in range(len(vormid)):
    if "käskiv" in vormid[j][0]:
    #kui leidub vähemalt üks sõna käskivas kõneviisis, siis on soovitus
      tulemus[i] = ("soovitus")
      break
  if tulemus[i] == ("soovitus"):
    continue
  tulemus[i] = ("kirjeldav") #kõigil muudel juhtudel on tegemist kirjeldusega

### 3. Meelsusanalüüs

In [8]:
#Fikseeritud suurusega (korpuse pikkusega) järjend tulemuste hoidmiseks
meelsused = np.empty(len(df), dtype=object)
loeng = np.empty(len(df), dtype=object)
eksam = np.empty(len(df), dtype=object)
õppejõud = np.empty(len(df), dtype=object)

for i in range(len(df)):
#vaadatakse ükshaaval läbi kõik andmeread
    kommentaar = df["tagasiside"][i]
    tekst = Text(kommentaar).tag_layer(['words', 'morph_analysis_est'])
    #morfoloogiline analüüs, sõnestamine
    skoor = 0 #skoori arvutamiseks iga kommentaari kohta
    eitus = 0 #indeks, kust alates otsitake eitust
    arvutus = False #kas lausest on leitud meelsussõna
    for k in range(len(tekst.words)): 
    #vaadatakse ükshaaval läbi kõik kommentaari sõned
        if len(tekst.words[k].text) < 2: #sõnede pikkuste kontroll
            continue
        for j in range(len(leksikon)):
        #vaadatakse läbi kõik valentsisõnastiku sõnad
            if tekst.words[k].text.lower() == leksikon["valjend"][j]:
            #korpuse sõne võrdlus valentsisõnastiku sõnaga
                ei = False #kas sõna kohta käib eitus
                eituseTekst = [word.lower() for word in tekst.words[eitus:min(k+2,len(tekst.words))].text]
                #vahemik millest eitust otsitakse
                if "ei" in eituseTekst or "mitte" in eituseTekst or "pole" in eituseTekst or "polnud" in eituseTekst or "ega" in eituseTekst:
                #eituse otsimine
                    skoor -= leksikon["hinnang"][j]#eitus, lahutatakse väärtus
                    eitus = k
                    ei = True
                else:
                    skoor += leksikon["hinnang"][j] #eitust pole, liidetakse
                    arvutus = True

                #süntaksipuu loomine
                text = Text(kommentaar).tag_layer()
                conll_tagger = ConllMorphTagger(output_layer='conll_morph',morph_extended_layer='morph_analysis',no_visl=True)
                conll_tagger.tag(text)
                maltparser_tagger = MaltParserTagger(input_type='morph_analysis', version='conllu')
                maltparser_tagger.tag(text)
                puu = text.maltparser_syntax
                #tunnuste otsimine süntaksipuu otsestest naabritest
                if puu[k].parent_span:
                    if puu[k].parent_span.lemma == "loeng":
                        if not loeng[i]:
                            loeng[i] = []
                        if ei:
                            loeng[i].append("-" + puu[k].lemma)
                        else:
                            loeng[i].append(puu[k].lemma)
                    elif puu[k].parent_span.lemma == "eksam":
                        if not eksam[i]:
                            eksam[i] = []
                        if ei:
                            eksam[i].append("-" + puu[k].lemma)
                        else:
                            eksam[i].append(puu[k].lemma)
                    elif puu[k].parent_span.lemma == "õppejõud":
                        if not õppejõud[i]:
                            õppejõud[i] = []
                        if ei:
                            õppejõud[i].append("-" + puu[k].lemma)
                        else:
                            õppejõud[i].append(puu[k].lemma)
                for child in puu[k].children:
                    if child.lemma == "loeng":
                        if not loeng[i]:
                            loeng[i] = []
                        if ei:
                            loeng[i].append("-" + puu[k].lemma)
                        else:
                            loeng[i].append(puu[k].lemma)
                    if child.lemma == "eksam":
                        if not eksam[i]:
                            eksam[i] = []
                        if ei:
                            eksam[i].append("-" + puu[k].lemma)
                        else:
                            eksam[i].append(puu[k].lemma)
                    if child.lemma == "õppejõud":
                        if not õppejõud[i]:
                            õppejõud[i] = []
                        if ei:
                            õppejõud[i].append("-" + puu[k].lemma)
                        else:
                            õppejõud[i].append(puu[k].lemma)
                break                    
    #meelsuse lõplik määramine
    if skoor > 0:
        meelsused[i] = "positiivne"
    elif skoor < 0:
        meelsused[i] = "negatiivne"
    elif skoor == 0 and arvutus == True:
        meelsused[i] = "vastuoluline"
    else:
        meelsused[i] = "neutraalne"